In [ ]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import json

my_session = sagemaker.Session()

role = get_execution_role()

s3 = boto3.resource('s3')
bucket_name = 'sg-ml-bucket-02'       # Change to your bucket
prefix = 'language_identification/fasttext' # and prefix

In [ ]:
my_region_name = boto3.Session().region_name

In [ ]:
blazing_text_container = sagemaker.amazon.amazon_estimator.get_image_uri(my_region_name, "blazingtext", "latest")
print('Using SageMaker BlazingText container: {} ({})'.format(blazing_text_container, my_region_name))

In [ ]:
!wget -O model.bin https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin

In [ ]:
#Tar the Language Identification and load to S3 bucket 
!tar -czvf langid.tar.gz model.bin
blazing_text_model_location = my_session.upload_data("langid.tar.gz", bucket=bucket_name, key_prefix=prefix)
!rm langid.tar.gz model.bin

In [ ]:
#Setup inference endpoint for model inference analysis
language_identifier = sagemaker.Model(model_data=blazing_text_model_location, image_uri=blazing_text_container, role=role, sagemaker_session=my_session)


In [ ]:
language_identifier.deploy(initial_instance_count = 1,instance_type = 'ml.m5.xlarge')

from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import JSONSerializer

language_identifier_predictor = sagemaker.Predictor(endpoint_name=language_identifier.endpoint_name, 
                                   sagemaker_session=my_session,
                                   serializer=JSONSerializer(), #json.dumps,
                                   deserializer=JSONDeserializer()) #sagemaker.predictor.json_deserializer)

In [ ]:
some_language_examples = ["À quoi sert l'intelligence artificielle",
             "Was ist der Zweck der künstlichen Intelligenz?",
             "Wat is die doel van kunsmatige intelligensie",
             "ما هو الغرض من الذكاء الاصطناعي",
             "Süni intellektin məqsədi nədir",
             "Hvad er formålet med kunstig intelligens"]
prediction_input = {"instances" : some_language_examples}

In [ ]:
language_predictions = language_identifier_predictor.predict(prediction_input)
print(language_predictions)

In [ ]:
# Remove the '__label__' before each language identifier in the prediction output
# and change the label and prob to more readable values
for output in language_predictions:
    output['label'] = output['label'][0][9:].upper() # remove __label__ preceding the language identifier
    output['language'] = output.pop('label')         # make the labels 
    output['probability'] = output.pop('prob')       # readable

print(language_predictions)